Transformers provides a `Trainer` object to help in fine-tuning any of the pretrained models it provides on your dataset.

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

In [3]:
raw_dataset = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [4]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
def tokenize_function(sample):
  return tokenizer(sample["sentence1"], sample["sentence2"], truncation=True)

In [6]:
tokenized_datasets = raw_dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

### Training

In [7]:
from transformers import TrainingArguments

In [8]:
training_args = TrainingArguments("trainer")

Defining the model

In [9]:
from transformers import AutoModelForSequenceClassification

In [10]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Now, we define `Trainer` by passing all the objects constructed up to now.

In [11]:
from transformers import Trainer

In [12]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer
)

In [13]:
trainer.train()

Step,Training Loss
500,0.506700
1000,0.249000


TrainOutput(global_step=1377, training_loss=0.30210025984044514, metrics={'train_runtime': 236.2031, 'train_samples_per_second': 46.587, 'train_steps_per_second': 5.83, 'total_flos': 405114969714960.0, 'train_loss': 0.30210025984044514, 'epoch': 3.0})

### Evaluation

Now, we'll create predictions for the model we built, and for that, we will use the `Trainer.predict()` command.

In [14]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

(408, 2) (408,)


The output of the `predict()` method is a named tuple containing three fields:
* **predictions**
* **label_ids**, and
* **metrics**

We can see from the above output that we need to convert the logits returned by the model to transform them into predictions.

In [15]:
import numpy as np

In [16]:
preds = np.argmax(predictions.predictions, axis=-1)

It's time to evaluate the model

In [17]:
import evaluate

In [18]:
metric = evaluate.load("glue", "mrpc")
metric.compute(
    predictions=preds,
    references=predictions.label_ids
)

{'accuracy': 0.8480392156862745, 'f1': 0.8949152542372881}

Here's time to wrap everything together.

In [26]:
def compute_metrics(eval_predictions):

  metric = evaluate.load("glue", "mrpc")
  logits, labels = eval_predictions
  preds = np.argmax(logits, axis=-1)

  return metric.compute(
      predictions=preds,
      references=labels
  )


Now, we'll define `Trainer` with functionality of reporting metrucs at the end of each epoch.

In [27]:
training_args = TrainingArguments(
                                  "trainer",
                                  evaluation_strategy="epoch"
                              )

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [28]:
model = AutoModelForSequenceClassification.from_pretrained(
                                                checkpoint,
                                                num_labels=2
                                            )

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Now that we have defined the `Trainer` with passing additional arguments of evaluation strategy and compute_metric, we execute new training.

In [30]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.417862,0.821078,0.875639
2,0.542100,0.450163,0.845588,0.891566
3,0.340800,0.662934,0.843137,0.889273


TrainOutput(global_step=1377, training_loss=0.3811595143835529, metrics={'train_runtime': 230.8587, 'train_samples_per_second': 47.666, 'train_steps_per_second': 5.965, 'total_flos': 405114969714960.0, 'train_loss': 0.3811595143835529, 'epoch': 3.0})